In [1]:
setwd(fs::path_abs("~/Local_Workspace/TesiMag"))
Sys.setlocale("LC_ALL", "UTF-8")
library(dplyr, warn.conflicts = FALSE)
library(openxlsx, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(stringi, warn.conflicts = FALSE)

source("src/database/startup.R")
source("src/database/query/data.R")

conns <- load_dbs()


[1] "C/UTF-8/C/C/C/C"

In [7]:
rg <- "LOM"
meta <- query_checkpoint_meta(rg, "merged", conns$data) |> collect()
corrections <- read.xlsx(fs::path("external", "correzioni", paste0(rg, ".xlsx"))) |>
    mutate(name = stri_unescape_unicode(str_replace_all(name, regex("<U\\+(.{4})>"), paste0("\\\\u", "\\1"))))

In [3]:
corrections |>
    mutate(across(c(from_sensor_keys, from_datasets), ~ str_split(., fixed("*"), simplify = FALSE))) |>
    select(name, from_sensor_keys, from_datasets) |>
    inner_join(meta, by = c("from_sensor_keys", "from_datasets"))

Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


Warning message in cbind(parts$left, chars$ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


name.x,from_sensor_keys,from_datasets,sensor_key,sensor_id,station_key,station_id,name.y,network,town,...,lon,lat,elevation,elevation_glo30,series_id,dataset,series_first,series_last,valid_days,valid90
<chr>,<list>,<list>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,...,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<date>,<date>,<int>,<int>


In [11]:
meta |>
    rowwise() |>
    mutate(across(starts_with("from_"), ~ paste0(., collapse="_"))) |>
    ungroup() |>
    full_join(corrections, by = "sensor_key", suffix = c(".now", ".manual")) |>
    filter(from_datasets.now != from_datasets.manual | from_sensor_keys.now != from_sensor_keys.manual | is.na(from_sensor_keys.now) | is.na(from_sensor_keys.manual)) |>
    select(name.now, name.manual, from_sensor_keys.now, from_sensor_keys.manual) |>
    as.data.frame()

name.now,name.manual,from_sensor_keys.now,from_sensor_keys.manual
<chr>,<chr>,<chr>,<chr>
